<a id="top"></a>

# Two Phase Flow Examples

<a id = "table"></a>

## Examples

[Example 1. One injector, one producer well.](#ex1)

<img src=".\images\2ph_ex1_sw.gif" width=400>
<img src=".\images\2ph_ex1_po.gif" width=400>

[Example 2. Two injectors, one producer well, and a low permeability region.](#ex2)

<img src=".\images\2ph_ex2_sw.gif" width=400>
<img src=".\images\2ph_ex2_po.gif" width=400>

<a id = "ex1"></a>

In [1]:
from modules.simulator.simulator import *
from modules.simulator.two_phase_flow import *
from modules.simulator.simplots import *
from modules.simulator.units import UnitRegistry
import numpy as np

# Set image folder path
imgpath = ".\\images"
# Units
u = UnitRegistry()

## [Example 1. One injector and  one producer well.](#table)


Two  layers with 10 by 10 cells, with one water injector in the bottom layer and one oil producer in the top layer. The simulation is run for 110 days with the Fully Implicit Method.

In [2]:
# GRID
Nx, Ny, Nz = np.array([10, 10, 2])
Sx, Sy, Sz = np.array([30, 30, 30]) * u.feet

depth = np.hstack((Sz * 0 * np.ones([Nx * Ny, ]), 
                  Sz * 1 * np.ones([Nx * Ny, ])))
                
grid = uniformCartesianGrid(Nx, Ny, Nz, Sx, Sy, Sz, depth)

# ROCK
perm = 100 * np.ones([grid.cellnumber, ]) * u.milli * u.darcy
poro = 0.15 * np.ones([grid.cellnumber, ])
# Rock compressibility equal for all cells
cr = 3E-6 / u.psi
porofunc = lambda p: poro * np.exp(cr * (p - 2800 * u.psi))
rock = Rock(perm, poro, cr, porofunc)

#  FLUID
# Oil phase
cf_o = 1E-5 / u.psi
miu_o = lambda p: ( -6E-5 * (p / u.psi) + 1.1791 ) * u.centi * u.poise
rho_o = lambda p: 45 * np.exp(cf_o * (p - 2800 * u.psi)) * u.pound / u.feet**3
fvf_o = lambda p : -1E-5 * (p / u.psi) + 1.018
# Water phase
cf_w = 3E-6 / u.psi
miu_w = lambda p: 1 * np.exp(5E-5 * (p / u.psi - 2800)) * u.centi * u.poise
#miu_w = lambda p: (1 +  5E-5 * (p / u.psi - 2800)) * u.centi * u.poise
rho_w = lambda p: 62.4 * np.exp(cf_w * (p - 2800 * u.psi)) * u.pound / u.feet**3
fvf_w = lambda p: 1 * np.exp(-3E-6 * (p / u.psi - 2800 ))   # adimensional

# Relative permeability functions
Swr = 0.25
Sor = 0.30
Swmin = Swr
Somin = Sor
Korw = 0.08
Koro = 0.7
Swmax = 1 - Somin
Somax = 1 - Swmin

kr_o = lambda So: Koro * ((So - Sor) / (1 - Sor - Swr)) ** 3
kr_w = lambda Sw: Korw * ((Sw - Swr) / (1 - Sor - Swr)) ** 2


dkr_o = lambda So: 3 * Koro  * (So - Sor) ** 2 / (1 - Sor - Swr) ** 3
dkr_w = lambda Sw: 2 * Korw * (Sw - Swr) / (1 - Sor - Swr) ** 2

def krwfunc(Sw):
    ''' Water relative permeambility
    Argument: Sw (np.array)'''
    kr = np.zeros_like(Sw)
    imin = Sw <= Swmin
    imax = Sw >= Swmax
    i = np.logical_not(np.logical_or(imin, imax))
    # The array is already filled with zeros    
    # kr[imin] = 0 
    kr[i] = kr_w(Sw[i])
    kr[imax] = Korw
    return kr

def krofunc(Sw):
    ''' Oil relative permeambility
    Argument: Sw (np.array)'''
    So = 1 - Sw
    kr = np.zeros_like(So)
    imin = So <= Somin
    imax = So >= Somax
    i = np.logical_not(np.logical_or(imin, imax))
    # The array is already filled with zeros
    # kr[imin] = 0
    kr[i] = kr_o(So[i])
    kr[imax] = Koro
    return kr


def d_krwfunc(Sw):
    ''' Water relative permeambility
    Argument: Sw (np.array)'''
    kr = np.zeros_like(Sw)
    imin = Sw <= Swmin
    imax = Sw >= Swmax
    i = np.logical_not(np.logical_or(imin, imax))
    # The array is already filled with zeros    
    # kr[imin] = 0 
    kr[i] = dkr_w(Sw[i])
    
    return kr

def d_krofunc(Sw):
    ''' Oil relative permeambility
    Argument: Sw (np.array)'''
    So = 1 - Sw
    kr = np.zeros_like(So)
    imin = So <= Somin
    imax = So >= Somax
    i = np.logical_not(np.logical_or(imin, imax))
    # The array is already filled with zeros
    # kr[imin] = 0
    kr[i] = dkr_o(So[i])
    
    return kr

# For this model, there is zero capillary pressure
pc = lambda Sw: 0 * Sw

fluid = blackOil((miu_o, miu_w), (rho_o, rho_w),
                 (fvf_o, fvf_w), (cf_o, cf_w), (krofunc, krwfunc), pc)

fluid.oilphase.d_kr = d_krofunc
fluid.waterphase.d_kr = d_krwfunc

# SCHEDULE
t1 = 0.1 * np.ones(100) * u.day
t2 = 0.5 * np.ones(100) * u.day
t3 = 1 * np.ones(50) * u.day
t4 = 2 * np.ones(50) * u.day
#t4 = 3 * np.ones(50) * u.day

timesteps = np.hstack((t1,t2,t3))
sch = Schedule(timesteps[:])

# BOUNDARY'S
boundary = Boundary()

# INITIAL CONDITIONS
p_init = 3000 * u.psi * np.ones([grid.cellnumber, ])
sw_init = 0.2 * np.ones([grid.cellnumber, ])

# WELLS
# Producer well
wells = Wells(grid, rock, fluid)
ci = Nx * Ny - 1
wells.add_vertical_well(0.35 * u.feet, ci, 2900 * u.psi, 0, 'Producer')
# Water injection well
source = np.zeros([grid.cellnumber * fluid.phasenumber, 1])
source[0] = -300 * u.barrel / u.day

# INITIALIZE SIMULATOR
FIM = ImplicitTwoPhase(
    grid, rock, fluid, wells, source, p_init, sw_init, boundary, gravity=True)

# RUN SIMULATION
results, well_solution, sch, info = FIM.solve(sch, max_iter= 10, tol = 1E-6, ATS = False)

  	 Newton-Raphson solver :  2/10. Error: 6.39E-14

In [4]:
# We will  save a plot for each timestep and then use 
# ffmpeg to create a gif
acum_sum = np.cumsum(sch.timesteps) / u.day
acum_sum = np.hstack((0, acum_sum))

#Transform results to psi
p_oil = results['p_oil'][:,:]/ u.psi
sw = results['sw'][:,:]

for k in np.arange(0, acum_sum.size, 1):
    plotCellValues2D(grid, p_oil[:, k], 'inferno', np.min(p_oil), np.max(p_oil),
                     title='Oil pressure. {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\2ph_ex1_po_{}'.format(imgpath,k))

for k in np.arange(0, acum_sum.size, 1):
    plotCellValues2D(grid, sw[:, k], 'plasma', np.min(sw), np.max(sw),
                     title='Water saturation. {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\2ph_ex1_sw_{}'.format(imgpath,k))

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


<a id="ex2"></a>

## [Example 2](#table)

For the second example we will use the same conditions as the [Example 1](#ex1), but we will add a second water injector. Also, we will modify the permeability of the lower layer, creating a zone of lower peameability.

In [7]:
# ROCK
perm = 100 * np.ones([grid.cellnumber, ]) * u.milli * u.darcy
# Low permeability cells
perm[[26, 36, 37, 48, 47]] = 1 *u.milli *u.darcy
poro = 0.15 * np.ones([grid.cellnumber, ])
# Rock compressibility equal for all cells
cr = 3E-6 / u.psi
porofunc = lambda p: poro * np.exp(cr * (p - 2800 * u.psi))
rock = Rock(perm, poro, cr, porofunc)


# WELLS
# Producer well
wells = Wells(grid, rock, fluid)
ci = Nx * (Ny-1) - 5
wells.add_vertical_well(0.35 * u.feet, ci, 2900 * u.psi, 0, 'Producer')

# Water injection well
source = np.zeros([grid.cellnumber * fluid.phasenumber, 1])
# Two water injectors  in cell 18 and cell 11.
# The water term is on (cellnumber * 2)
# The oil term is on (cellnumber * 2 + 1)
source[22] = -150 * u.barrel / u.day
source[36] = -170 * u.barrel / u.day

# INITIALIZE SIMULATOR
FIM = ImplicitTwoPhase(
    grid, rock, fluid, wells, source, p_init, sw_init, boundary, gravity=True)

# RUN SIMULATION
results, well_solution, sch, info = FIM.solve(sch, max_iter= 10, tol = 1E-6, ATS = False)

  	 Newton-Raphson solver :  2/10. Error: 5.62E-13

In [8]:
# Plot the rock permeability
permtoplot = np.log(rock.perm / u.darcy / u.milli)
plotCellValues2D(grid, permtoplot, 'viridis', 
                 np.min(permtoplot), np.max(permtoplot), 
                 title='Permeability Ln[mD]', filename = ".\\images\\2ph_ex2_perm")

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


In [9]:
# We will  save a plot for each timestep and then use 
# ffmpeg to create a gif
acum_sum = np.cumsum(sch.timesteps) / u.day
acum_sum = np.hstack((0, acum_sum))

#Transform results to psi
p_oil = results['p_oil'][:,:]/ u.psi
sw = results['sw'][:,:]

for k in np.arange(0, acum_sum.size, 1):
    plotCellValues2D(grid, p_oil[:, k], 'inferno', np.min(p_oil), np.max(p_oil),
                     title='Oil pressure. {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\2ph_ex2_po_{}'.format(imgpath,k))

for k in np.arange(0, acum_sum.size, 1):
    plotCellValues2D(grid, sw[:, k], 'plasma', np.min(sw), np.max(sw),
                     title='Water saturation. {:.3f} days'.format(acum_sum[k]), 
                     filename='{}\\2ph_ex2_sw_{}'.format(imgpath,k))

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


### [To the top](#top)

In [10]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Using the style sheet found here  Lorena Barba /* https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css */